# Spike Gadgets Ephys and Video Syncing

In [116]:
import re
import sys
from collections import defaultdict
import random
from random import randrange
import glob
import warnings
import os
import git
import bisect
import h5py

In [2]:
import numpy as np
import pandas as pd
import cv2
from IPython.display import Video
import matplotlib.pyplot as plt

In [3]:
# setting path
sys.path.append('../../src')

In [4]:
import trodes.read_exported

In [5]:
%matplotlib inline

In [6]:
np.random.seed(seed=42)

# Part 0: Index of all the column names

raw directory
- raw_group0.dat
    - voltage_value: Array with voltage measurement for each channel at each timestamp
- timestamps.dat
    - voltage_time_stamp: The time stamp of each voltage measurement

parent directory
- 1.videoTimeStamps.cameraHWSync
    - frame_number: Calculated by getting the index of each video time stamp tuple 
    - PosTimestamp: The time stamp of each video frame
    - HWframeCount: Unknown value. Starts at 30742 and increases by 1 for each tuple  
    - HWTimestamp: Unknown value. All zeroes
    - video_time: Calculated by dividing the frame number by the fps(frames per second) 
    - video_seconds: video_time, but rounded to seconds  	
    - These are filled in versions of the above collumns with the value from the most recent previous cell
        - filled_PosTimestamp 	
        - filledHWframeCount 	
        - filled_frame_number 	
        - filled_video_time 	
        - filled_video_seconds 	

DIO directory
- dio_ECU_Din1.dat
    - time: The time stamp the corresponds to the DIN input
    - state: Binary state of whether there is input from DIN or not 	
    - trial_number: Calculated by adding 1 to every time there is a DIN input
    - These are filled in versions of the above collumns with the value from the most recent previous cell
        - filled_state 	
        - filled_trial_number

ss_output directory (Spike sorting with Spike interface)
- firings.npz
    - unit_id: All the units that had a spike train for the given timestamp 	
    - number_of_units: Calculated by counting the number of units that had a spike train

# Part 1: Inputting Data

## Name of protocol for naming

- This name will be used to name files and title plots. Please change if you are using a different protocol or adding more details
    - **NOTE**: This should be changed based on the name the protocol

In [7]:
protocol_name = "rc_extention"

## Getting the file name of the raw data

- Default input folder and keyword to search the files for 
    - **NOTE**: This should not be changed unless there is a consistent change with the file naming convention

In [8]:
input_folder = "./data"

- Make this cell into non-code block if you are using the same file path for multiple runs

In [9]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [10]:
git_root

'/nancy/projects/reward_competition_ephys_analysis_with_omission_and_divider_controls'

In [11]:
recording_filepath_glob = "data/test"

In [104]:
recording_filepath_glob = "data/good/*20221202*"

In [105]:
recording_absolute_path_glob = os.path.join(git_root, recording_filepath_glob)

In [106]:
# Getting all the file paths of the recording files(that happen to all end in `.rec`)
raw_data_all_files = glob.glob(recording_absolute_path_glob, recursive=True)

In [107]:
raw_data_all_files

['/nancy/projects/reward_competition_ephys_analysis_with_omission_and_divider_controls/data/good/20221202_134600_omission_and_competition_subject_6_1_and_6_2.rec']

## Extracting the data and the metadata from the Recording folder

- Creating a dictionary that has the directory as the key and a dictionary that has the file name as the key and the 

In [108]:

def find_closest(my_list, my_number):
    """
    Assumes my_list is sorted. Returns the closest value to my_number.

    If two numbers are equally close, return the smallest number.
    """
    pos = bisect.bisect_left(my_list, my_number)
    if pos == 0:
        return my_list[0]
    else:
        return my_list[pos - 1]

In [111]:
raw_data_file_path = raw_data_all_files[0]

In [112]:
raw_data_file_path

'/nancy/projects/reward_competition_ephys_analysis_with_omission_and_divider_controls/data/good/20221202_134600_omission_and_competition_subject_6_1_and_6_2.rec'

In [113]:
raw_data_dir_to_extracted_files = defaultdict(dict)
# Getting the basename of the recording
recording_dirname = os.path.basename(raw_data_file_path)
recording_basename = os.path.splitext(recording_dirname)[0]
# Extracting the files
file_to_data = trodes.read_exported.get_all_trodes_data_from_directory(raw_data_file_path)


/nancy/projects/reward_competition_ephys_analysis_with_omission_and_divider_controls/results/2023_05_23_exporting_data/../../src/trodes/read_exported.py:70: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  return np.dtype(typearr)


file prefix: group0.coordinates.dat
directory prefix: raw
file prefix: timestamps.dat
directory prefix: raw
file prefix: raw_group0.dat
directory prefix: raw
file prefix: 1.videoTimeStamps.cameraHWSync
directory prefix: .
file prefix: 1.videoTimeStamps.cameraHWSync
directory prefix: .


/nancy/projects/reward_competition_ephys_analysis_with_omission_and_divider_controls/results/2023_05_23_exporting_data/../../src/trodes/read_exported.py:138: UserWarning: Can not process /nancy/projects/reward_competition_ephys_analysis_with_omission_and_divider_controls/data/good/20221202_134600_omission_and_competition_subject_6_1_and_6_2.rec/20221202_134600_omission_and_competition_subject_6_1_top_2_base_3_merged.timestampoffset/20221202_134600_omission_and_competition_subject_6_1_top_2_base_3_merged.timestampoffset.txt
  warnings.warn("Can not process {}".format(absolute_file_path))
/nancy/projects/reward_competition_ephys_analysis_with_omission_and_divider_controls/results/2023_05_23_exporting_data/../../src/trodes/read_exported.py:138: UserWarning: Can not process /nancy/projects/reward_competition_ephys_analysis_with_omission_and_divider_controls/data/good/20221202_134600_omission_and_competition_subject_6_1_and_6_2.rec/20221202_134600_omission_and_competition_subject_6_1_and_6_

file prefix: analog_Headstage_GyroZ.dat
directory prefix: analog
file prefix: analog_ECU_Ain6.dat
directory prefix: analog
file prefix: analog_ECU_Aout1.dat
directory prefix: analog
file prefix: analog_Headstage_MagY.dat
directory prefix: analog
file prefix: analog_ECU_Ain8.dat
directory prefix: analog
file prefix: timestamps.dat
directory prefix: analog
file prefix: analog_Headstage_MagX.dat
directory prefix: analog
file prefix: analog_Headstage_AccelY.dat
directory prefix: analog
file prefix: analog_ECU_Ain3.dat
directory prefix: analog
file prefix: analog_Headstage_GyroX.dat
directory prefix: analog
file prefix: analog_ECU_Ain2.dat
directory prefix: analog
file prefix: analog_ECU_Aout4.dat
directory prefix: analog
file prefix: analog_Headstage_AccelX.dat
directory prefix: analog
file prefix: analog_ECU_Aout3.dat
directory prefix: analog
file prefix: analog_ECU_Ain1.dat
directory prefix: analog
file prefix: analog_ECU_Ain4.dat
directory prefix: analog
file prefix: analog_ECU_Ain7.dat

# Part 2: Looking over the data

## Looking over the ephys recording

- Getting the name of the ephys recording directory

In [147]:
file_to_data 

defaultdict(dict,
            {'20221202_134600_omission_and_competition_subject_6_1_top_2_base_3_merged.raw': {'group0.coordinates.dat': {'description': 'Pad locations in microns',
               'byte_order': 'little endian',
               'original_file': '20221202_134600_omission_and_competition_subject_6_1_top_2_base_3_merged.rec',
               'clockrate': '20000',
               'trodes_version': '2.3.2',
               'compile_date': 'Apr 12 2022',
               'compile_time': '15:21:02',
               'qt_version': '6.2.2',
               'commit_tag': 'heads/Release_2.3.2-0-g15f12712',
               'controller_firmware': '3.17',
               'headstage_firmware': '2.2',
               'controller_serialnum': '00104 00176',
               'headstage_serialnum': '01601 00130',
               'autosettle': '0',
               'smartref': '0',
               'gyro': '0',
               'accelerometer': '0',
               'magnetometer': '1',
               'time_offse

In [148]:
raw_directory_dict = file_to_data["raw"]

- Getting the name of the files in the ephys recording directory

In [149]:
raw_directory_dict.keys()

dict_keys(['group0.coordinates.dat', 'timestamps.dat', 'raw_group0.dat'])

In [150]:
raw_recording_fields_text = raw_directory_dict["raw_group0.dat"]

In [151]:
raw_recording_fields_text

{'description': 'Raw (unfiltered) data for one sorting group',
 'byte_order': 'little endian',
 'original_file': '20221202_134600_omission_and_competition_subject_6_1_top_2_base_3_merged.rec',
 'clockrate': '20000',
 'trodes_version': '2.3.2',
 'compile_date': 'Apr 12 2022',
 'compile_time': '15:21:02',
 'qt_version': '6.2.2',
 'commit_tag': 'heads/Release_2.3.2-0-g15f12712',
 'controller_firmware': '3.17',
 'headstage_firmware': '2.2',
 'controller_serialnum': '00104 00176',
 'headstage_serialnum': '01601 00130',
 'autosettle': '0',
 'smartref': '0',
 'gyro': '0',
 'accelerometer': '0',
 'magnetometer': '1',
 'time_offset': '0',
 'system_time_at_creation': '1670006785156',
 'timestamp_at_creation': '522374',
 'first_timestamp': '4919837',
 'sorting_group': '0',
 'num_channels': '32',
 'voltage_scaling': '0.195',
 'fields': '<voltage 32*int16>',
 'data': array([([-310, -138, -329, -435, -217, -393, -281, -253, -246, -190,  -56,  -35, -116,  284, 1023,  696,  417,  -92, -213, -258, -153

- Voltage Time Stamps

In [152]:
raw_directory_dict.keys()

dict_keys(['group0.coordinates.dat', 'timestamps.dat', 'raw_group0.dat'])

In [153]:
voltage_timestamp_array = raw_directory_dict["timestamps.dat"]["data"]

In [154]:
voltage_timestamp_array[:5]

array([(4919837,), (4919838,), (4919839,), (4919840,), (4919841,)],
      dtype=[('time', '<u4')])

In [155]:
voltage_timestamp_array[-5:]

array([(71918310,), (71918311,), (71918312,), (71918313,), (71918314,)],
      dtype=[('time', '<u4')])

In [156]:
voltage_timestamp_array.shape

(66998478,)

- Converting the array to integers to be able to do calculations

    - u4 and i2 explanation: https://www.geeksforgeeks.org/data-type-object-dtype-numpy-python/

In [157]:
voltage_timestamp_array = voltage_timestamp_array.astype(int)

In [158]:
voltage_timestamp_array[:5]

array([4919837, 4919838, 4919839, 4919840, 4919841])

In [159]:
voltage_timestamp_array[-5:]

array([71918310, 71918311, 71918312, 71918313, 71918314])

In [160]:
voltage_timestamp_array.shape

(66998478,)

# Exporting Video

## Looking over the video files

In [161]:
parent_directory_dict = file_to_data["."]

In [162]:
video_time_stamp_dict = parent_directory_dict["1.videoTimeStamps.cameraHWSync"]

In [163]:
video_time_stamp_dict

{'clock rate': '30000',
 'fields': '<PosTimestamp uint32><HWframeCount uint32><HWTimestamp uint64>',
 'data': array([( 4919835, 0, 0), ( 4921221, 0, 0), ( 4921221, 0, 0), ...,
        (71916928, 0, 0), (71918314, 0, 0), (71918314, 0, 0)],
       dtype=[('PosTimestamp', '<u4'), ('HWframeCount', '<u4'), ('HWTimestamp', '<u8')]),
 'absolute_file_path': '/nancy/projects/reward_competition_ephys_analysis_with_omission_and_divider_controls/data/good/20221202_134600_omission_and_competition_subject_6_1_and_6_2.rec/20221202_134600_omission_and_competition_subject_6_1_and_6_2.1.videoTimeStamps.cameraHWSync'}

In [164]:
video_time_stamp_dict["data"]

array([( 4919835, 0, 0), ( 4921221, 0, 0), ( 4921221, 0, 0), ...,
       (71916928, 0, 0), (71918314, 0, 0), (71918314, 0, 0)],
      dtype=[('PosTimestamp', '<u4'), ('HWframeCount', '<u4'), ('HWTimestamp', '<u8')])

In [165]:
video_time_stamp_dict["data"][:5]

array([(4919835, 0, 0), (4921221, 0, 0), (4921221, 0, 0), (4922607, 0, 0),
       (4922607, 0, 0)],
      dtype=[('PosTimestamp', '<u4'), ('HWframeCount', '<u4'), ('HWTimestamp', '<u8')])

In [166]:
video_time_stamp_dict["data"][-5:]

array([(71915542, 0, 0), (71915542, 0, 0), (71916928, 0, 0),
       (71918314, 0, 0), (71918314, 0, 0)],
      dtype=[('PosTimestamp', '<u4'), ('HWframeCount', '<u4'), ('HWTimestamp', '<u8')])

In [167]:
video_time_stamp_dict["data"].shape

(83387,)

In [168]:
video_time_stamp_array = video_time_stamp_dict["data"]["PosTimestamp"].astype(int) - voltage_timestamp_array[0] + 1

In [169]:
video_time_stamp_array[video_time_stamp_array <= 0] = 0

In [170]:
video_time_stamp_array

array([       0,     1385,     1385, ..., 66997092, 66998478, 66998478])

In [208]:
video_df = pd.DataFrame(video_time_stamp_array, columns=["time_stamp"]).reset_index()
video_df.rename(columns={"index": "video_frame"})

,video_frame,time_stamp
0,0,0
1,1,1385
2,2,1385
3,3,2771
4,4,2771
...,...,...
83382,83382,66995706
83383,83383,66995706
83384,83384,66997092
83385,83385,66998478


# Exporting SLEAP

# Getting the coordinates of each mouse to the reward port

In [209]:
filename_1_subj = "./proc/sleap/20221202_134600_omission_and_competition_subject_6_1_and_6_2.1.fixed.1_subj.round_5.analysis.h5"
filename_2_subj = "./proc/sleap/20221202_134600_omission_and_competition_subject_6_1_and_6_2.1.fixed.2_subj.round_5.analysis.h5"

## Loading the data

We use the [h5py](https://www.h5py.org) package to load data from the HDF5. This is already installed in Colab. If your running analysis code on your local machine and have SLEAP installed, then `h5py` and other packages we use are already installed in your SLEAP conda environment. Otherwise, you may need to use `conda` or `pip` to install `h5py` as well as `numpy`, `scipy`, `matplotlib`, `seaborn`, and any other packages you want use in your analysis code.

Let's load the file and take a peek.

In [210]:
with h5py.File(filename_2_subj, "r") as f:
    dset_names = list(f.keys())
    locations = f["tracks"][:].T
    node_names = [n.decode() for n in f["node_names"][:]]
    track_names = f["track_names"][:].T
    track_names = [str(name.decode('UTF-8')) for name in track_names]

print("===filename===")
print(filename_2_subj)
print()

print("===HDF5 datasets===")
print(dset_names)
print()

print("===locations data shape===")
print(locations.shape)
print()

print("===nodes===")
for i, name in enumerate(node_names):
    print(f"{i}: {name}")
print()


===filename===
./proc/sleap/20221202_134600_omission_and_competition_subject_6_1_and_6_2.1.fixed.2_subj.round_5.analysis.h5

===HDF5 datasets===
['edge_inds', 'edge_names', 'instance_scores', 'labels_path', 'node_names', 'point_scores', 'provenance', 'track_names', 'track_occupancy', 'tracking_scores', 'tracks', 'video_ind', 'video_path']

===locations data shape===
(83387, 6, 2, 113)

===nodes===
0: left_ear
1: right_ear
2: nose
3: tail_base
4: thorax
5: forehead



In [211]:
track_names

['6_1',
 '6_2',
 'track_273',
 'track_274',
 'track_275',
 'track_276',
 'track_278',
 'track_279',
 'track_280',
 'track_281',
 'track_282',
 'track_288',
 'track_289',
 'track_290',
 'track_291',
 'track_292',
 'track_300',
 'track_301',
 'track_302',
 'track_303',
 'track_304',
 'track_305',
 'track_306',
 'track_307',
 'track_310',
 'track_311',
 'track_314',
 'track_315',
 'track_317',
 'track_318',
 'track_319',
 'track_320',
 'track_323',
 'track_324',
 'track_325',
 'track_326',
 'track_327',
 'track_328',
 'track_329',
 'track_330',
 'track_331',
 'track_332',
 'track_334',
 'track_335',
 'track_338',
 'track_339',
 'track_346',
 'track_347',
 'track_351',
 'track_352',
 'track_355',
 'track_356',
 'track_357',
 'track_358',
 'track_359',
 'track_360',
 'track_361',
 'track_362',
 'track_365',
 'track_366',
 'track_367',
 'track_368',
 'track_370',
 'track_371',
 'track_373',
 'track_374',
 'track_376',
 'track_377',
 'track_378',
 'track_379',
 'track_381',
 'track_382',
 'tr

In [212]:
locations = locations[:,:,:,:2]

In [213]:
locations.shape

(83387, 6, 2, 2)

In our example file, the shape of the locations matrix (the `tracks` dataset) is (3000, 13, 2, 2) **after it is transposed** (with the `.T`). We transpose the data when loading it in Python; no transpose is needed when using MATLAB. This is because Python and MATLAB expect matrices to be stored differently.

Here's what each dimension of the matrix means:

- 3000: the number of frames;

- 13: the number of nodes in the skeleton (we've also loaded and displayed the `node_names` dataset with the names of these 13 nodes);

- 2: for the x and y coordinates;

- 2: the number of distinct animal identities which were found (we have 2 flies in the video clip and they were tracked perfectly, so we ended up with exactly 2 track, but there may be more tracks than animals if tracking didn't work as well).

We can get these counts from the shape of the matrix, like so:


In [214]:
frame_count, node_count, _, instance_count = locations.shape

print("frame count:", frame_count)
print("node count:", node_count)
print("instance count:", instance_count)

frame count: 83387
node count: 6
instance count: 2


Now that we've loaded the data, let's see some different things we can do with it...

## Fill missing values

In [215]:
from scipy.interpolate import interp1d

def fill_missing(Y, kind="linear"):
    """Fills missing values independently along each dimension after the first."""

    # Store initial shape.
    initial_shape = Y.shape

    # Flatten after first dim.
    Y = Y.reshape((initial_shape[0], -1))

    # Interpolate along each slice.
    for i in range(Y.shape[-1]):
        y = Y[:, i]

        # Build interpolant.
        x = np.flatnonzero(~np.isnan(y))
        f = interp1d(x, y[x], kind=kind, fill_value=np.nan, bounds_error=False)

        # Fill missing
        xq = np.flatnonzero(np.isnan(y))
        y[xq] = f(xq)
        
        # Fill leading or trailing NaNs with the nearest non-NaN values
        mask = np.isnan(y)
        y[mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), y[~mask])

        # Save slice
        Y[:, i] = y

    # Restore to initial shape.
    Y = Y.reshape(initial_shape)

    return Y

In [216]:
locations = fill_missing(locations)

In [217]:
locations.shape

(83387, 6, 2, 2)

In [218]:
node_index

5

In [219]:
for track_index, track in enumerate(track_names[:2]):
    for node_index, node in enumerate(node_names):
        print("{} {}".format(track, node))
        print(locations[:,node_index,:,track_index].shape)
        video_df["{}_{}_x".format(track, node)] = locations[:,node_index,0,track_index]
        video_df["{}_{}_y".format(track, node)] = locations[:,node_index,1,track_index]


6_1 left_ear
(83387, 2)
6_1 right_ear
(83387, 2)
6_1 nose
(83387, 2)
6_1 tail_base
(83387, 2)
6_1 thorax
(83387, 2)
6_1 forehead
(83387, 2)
6_2 left_ear
(83387, 2)
6_2 right_ear
(83387, 2)
6_2 nose
(83387, 2)
6_2 tail_base
(83387, 2)
6_2 thorax
(83387, 2)
6_2 forehead
(83387, 2)


In [220]:
video_df

,index,time_stamp,6_1_left_ear_x,6_1_left_ear_y,6_1_right_ear_x,6_1_right_ear_y,6_1_nose_x,6_1_nose_y,6_1_tail_base_x,6_1_tail_base_y,...,6_2_right_ear_x,6_2_right_ear_y,6_2_nose_x,6_2_nose_y,6_2_tail_base_x,6_2_tail_base_y,6_2_thorax_x,6_2_thorax_y,6_2_forehead_x,6_2_forehead_y
0,0,0,799.845886,292.099457,832.411255,340.223450,859.966553,323.458282,732.571411,392.166779,...,868.722534,303.338898,1008.190674,877.336426,1207.559448,567.546692,868.238892,308.386383,820.080200,356.247009
1,1,1385,799.893738,291.766266,831.919067,343.731842,855.819214,323.988556,732.571411,392.166779,...,868.722534,303.338898,1008.190674,877.336426,1207.559448,567.546692,868.238892,308.386383,820.080200,356.247009
2,2,1385,800.135559,291.596497,831.896667,343.604370,855.844604,323.968506,732.571411,392.166779,...,868.722534,303.338898,1008.190674,877.336426,1207.559448,567.546692,868.238892,308.386383,820.080200,356.247009
3,3,2771,800.089478,291.393341,835.759949,336.076172,860.666138,316.348572,732.571411,392.166779,...,868.722534,303.338898,1008.190674,877.336426,1207.559448,567.546692,868.238892,308.386383,820.080200,356.247009
4,4,2771,796.298157,291.368896,831.958679,332.382446,863.970703,312.493805,732.571411,392.166779,...,868.722534,303.338898,1008.190674,877.336426,1207.559448,567.546692,868.238892,308.386383,820.080200,356.247009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83382,83382,66995706,1012.404724,871.534973,947.780334,864.115967,991.756042,932.195312,1047.578735,748.213257,...,691.666443,511.478851,736.327087,551.659363,752.010681,387.615906,719.622253,436.320831,716.072998,531.537109
83383,83383,66995706,1015.491150,868.183594,947.801453,864.096436,991.737244,932.146545,1047.596558,748.187073,...,691.731445,511.448395,736.327087,551.659363,752.025635,387.603851,719.628357,436.352203,712.456177,528.315002
83384,83384,66997092,1012.425171,868.249329,947.838501,864.092224,991.713257,932.078796,1047.593750,748.209717,...,691.774414,511.444092,736.327087,551.659363,752.045837,387.604401,719.649780,436.331421,712.546936,528.298584
83385,83385,66998478,1012.432556,868.244751,947.872803,864.114502,991.661194,932.149048,1047.595825,748.203735,...,691.794128,511.564514,736.327087,551.659363,752.042664,387.616272,716.570862,436.290283,712.535156,528.305542


In [184]:
track_names[:2]

['6_1', '6_2']

# Exporting LFP

# Exporting Spike Sorting

## Video time to Video frame

### Reading in video

In [ ]:
raw_data_file_path

In [ ]:
video_file_list = glob.glob(os.path.join("../../../../data/good/20221202_134600_omission_and_competition_subject_6_1_and_6_2.rec/", "*.mp4"))

In [ ]:
video_file_list

In [ ]:
video_file_path = video_file_list[0]

In [ ]:
# read video from file
cap = cv2.VideoCapture(video_file_path)

- Getting the number of frames per second

In [ ]:
fps = cap.get(cv2.CAP_PROP_FPS)

In [ ]:
fps

- Total number of frames

In [ ]:
total_frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)

In [ ]:
total_frame_count


- Calculating the length of the video by dividing the total number of frames by the fps

In [ ]:
video_length = total_frame_count / fps

In [ ]:
video_length

# Part 3: Converting everything into timestamps

## Voltage to timestamp

- Use the matching index to convert between the voltage time stamp and the recording

In [ ]:
voltage_index = 0

In [ ]:
voltage_value_array[voltage_index]

- Getting the associated timestamp of the ephys recording

In [ ]:
voltage_timestamp_array[:5]

In [ ]:
voltage_timestamp_array[-5:]

In [ ]:
voltage_time_stamp = voltage_timestamp_array[voltage_index]

In [ ]:
voltage_time_stamp

## Video frame to timestamp

- Getting the time stamps of the video frames
    - Each frame would correspond to each timestamp. Because the sample rate of videos is smaller than ephys recording, the number of video time stamps will be less than that of ephys recordings.

In [ ]:
video_time_stamp_dict

In [ ]:
video_time_stamp_array = np.array(video_time_stamp_dict["data"])

In [ ]:
video_time_stamp_array[:5]

In [ ]:
video_time_stamp_array[-5:]

In [ ]:
video_time_stamp_array.shape

- Getting only the first number in each tuple

In [ ]:
pos_timestamp_array = np.array([x[0] for x in video_time_stamp_array]) 

- Converting to integer to do calculations

In [ ]:
pos_timestamp_array = pos_timestamp_array.astype(int)

In [ ]:
pos_timestamp_array[:5]

In [ ]:
pos_timestamp_array[-5:]

# Part 4: Converting from timestamps back to everything

In [ ]:
def timestamp_to_index(current_ts, ts_array):
    """
    """
    return np.argwhere(ts_array >= current_ts)[0][0]

## Time stamp to Voltage Value

In [ ]:
voltage_timestamp_array

In [ ]:
voltage_timestamp_array[0]

- Getting the index of the closest timestamp 

In [ ]:
random_voltage_index = timestamp_to_index(current_ts=voltage_timestamp_array[0], ts_array=voltage_timestamp_array)

In [ ]:
random_voltage_index

- Checking if it matches to the original timestamp
    - Should be the same, because the sampling rate of the timestamps are based on the ephys recording

In [ ]:
voltage_timestamp_array[random_voltage_index]

- Getting the voltage value based on the index

In [ ]:
voltage_value_array[random_voltage_index]

## Time Stamp to Video Frame

In [ ]:
video_time_stamp_array

In [ ]:
pos_timestamp_array

- Getting the index of the closest timestamp. The index corresponds to the video's frame number

In [ ]:
random_video_frame = timestamp_to_index(current_ts=pos_timestamp_array[0], ts_array=pos_timestamp_array)

In [ ]:
random_video_frame

In [ ]:
pos_timestamp_array[random_video_frame]

# ADDED AFTER

# Part 4: Syncing everything based on timestamps

# Syncing with MED-PC

## Looking over the MED-PC Data

- Box 1 Port Entries
    - ECU Din3
- Box 2 Port Entries
    - Controller Din1
- Box 1 Tone playing
    - ECU Din1

In [ ]:
DIO_directory_name = trodes.read_exported.get_key_with_substring(file_to_data, substring="DIO")

In [ ]:
DIO_directory_name

In [ ]:
DIO_directory_dict = file_to_data[DIO_directory_name]

In [ ]:
DIO_directory_dict.keys()

### Tone Onset Signal

In [ ]:
tone_onset_DIN_file_name = trodes.read_exported.get_key_with_substring(DIO_directory_dict, substring="ECU_Din1.dat", return_first=True)

In [ ]:
tone_onset_DIN_file_name

In [ ]:
tone_onset_DIN_state_array = DIO_directory_dict[tone_onset_DIN_file_name]["data"]

In [ ]:
tone_onset_DIN_state_array

In [ ]:
plt.hist([tup[1] for tup in tone_onset_DIN_state_array])

In [ ]:
plt.plot([tup[0] for tup in tone_onset_DIN_state_array], [tup[1] for tup in tone_onset_DIN_state_array])
plt.xlabel("Timestamp")
plt.ylabel("State")
plt.title("Din State Change against Timestamps")

### Box 1 Port Entries

In [ ]:
box1_port_entries_DIN_file_name = trodes.read_exported.get_key_with_substring(DIO_directory_dict, substring="ECU_Din3.dat", return_first=True)

In [ ]:
box1_port_entries_DIN_file_name

In [ ]:
box1_port_entries_DIN_state_array = DIO_directory_dict[box1_port_entries_DIN_file_name]["data"]

In [ ]:
box1_port_entries_DIN_state_array

In [ ]:
plt.hist([tup[1] for tup in box1_port_entries_DIN_state_array])

- There is a gap in the middle when the protocol was being changed between competition or omission

In [ ]:
plt.plot([tup[0] for tup in box1_port_entries_DIN_state_array], [tup[1] for tup in box1_port_entries_DIN_state_array])
plt.xlabel("Timestamp")
plt.ylabel("State")
plt.title("Din State Change against Timestamps")

### Box 2 Port Entries

In [ ]:
box2_port_entries_DIN_file_name = trodes.read_exported.get_key_with_substring(DIO_directory_dict, substring="dio_Controller_Din1.dat", return_first=True)

In [ ]:
box2_port_entries_DIN_file_name

In [ ]:
box2_port_entries_DIN_file_name = DIO_directory_dict[box2_port_entries_DIN_file_name]["data"]

In [ ]:
box2_port_entries_DIN_file_name

In [ ]:
plt.hist([tup[1] for tup in box2_port_entries_DIN_file_name])

- This is half the time than the previous port entries, because the mouse was moved to box 1 for half of the session

In [ ]:
plt.plot([tup[0] for tup in box2_port_entries_DIN_file_name], [tup[1] for tup in box2_port_entries_DIN_file_name])
plt.xlabel("Timestamp")
plt.ylabel("State")
plt.title("Din State Change against Timestamps")

## Labeling the Tone and Port Entries

# Syncing with MED-PC

- List of when the ECU has changed signal. 1 means that the ECU Din1 signal is on, 0 means it's off.

In [ ]:
tone_onset_DIN_state_array

- Checking to see if 1 or 0 is when the tone plays
    - Dividing by 20000, because we are recording at a sampling rate at 20000

In [ ]:
tone_onset_DIN_state_array[0][0]

In [ ]:
tone_onset_DIN_state_array[0][1]

In [ ]:
tone_onset_DIN_state_array[1][0]

In [ ]:
tone_onset_DIN_state_array[1][1]

In [ ]:
tone_onset_DIN_state_array[2][0]

In [ ]:
tone_onset_DIN_state_array[2][1]

- So the tone starts when the state is "1"
    - This can be seen because the time from 1 to 0 is less than 60 seconds

In [ ]:
first_delay = (tone_onset_DIN_state_array[1][0] - tone_onset_DIN_state_array[0][0]) / 20000

In [ ]:
first_delay

- Time difference for on >>> off

In [ ]:
(tone_onset_DIN_state_array[3][0] - tone_onset_DIN_state_array[2][0]) / 20000

- So the tone starts when the state is "2"
    - This can be seen because the time from 0 to 1 is 60 seconds, the time for one session

In [ ]:
(tone_onset_DIN_state_array[2][0] - tone_onset_DIN_state_array[1][0]) / 20000

- Getting only the times when the ECU signal was on

In [ ]:
tone_din_time = [din_time for din_time, din_state in tone_onset_DIN_state_array if din_state == 1]

In [ ]:
len(tone_din_time)

In [ ]:
tone_din_time[:10]

# From DIN to Video

- State 1 is when the MED-PC signal is being recieved. And 0 is when it is turned off. So we will get the timestamp of when it is first 1.

In [ ]:
tone_onset_DIN_state_array

In [ ]:
tone_time_stamp = [time_state[0] for time_state in tone_onset_DIN_state_array if time_state[1]]

In [ ]:
example_tone_time_stamp = tone_time_stamp[3]

In [ ]:
example_tone_time_stamp

- Array of the time stamp of all the frames

In [ ]:
pos_timestamp_array

- Getting the first video time stamp that is greater than the voltage time stamp

In [ ]:
current_video_frame = timestamp_to_index(current_ts=example_tone_time_stamp, ts_array=pos_timestamp_array)

In [ ]:
current_video_frame

In [ ]:
timestamp_to_index(current_ts=28625643, ts_array=pos_timestamp_array)

# Syncing up the timestamps using Pandas

## Adding the Voltage as columns

In [ ]:
voltage_timestamp_array[:5]

In [ ]:
voltage_timestamp_array.shape

In [ ]:
voltage_value_array[:5]

In [ ]:
voltage_value_array.shape

- Adding the voltage timestamps

In [ ]:
ephys_dataframe = pd.DataFrame(voltage_timestamp_array, columns=["voltage_time_stamp"])

In [ ]:
ephys_dataframe.head()

- Adding the voltage value

In [ ]:
ephys_dataframe.head()

## Adding the video data as columns

- Creating a seperate dataframe for video data first

In [ ]:
pos_timestamp_array[:5]

In [ ]:
pos_timestamp_array[-5:]

In [ ]:
video_dataframe = pd.DataFrame(pos_timestamp_array, columns=["PosTimestamp"])

In [ ]:
video_dataframe.head()

- Adding the frames which would just be the number in the list that the timestamps belongs to

In [ ]:
video_dataframe.insert(0, 'frame_number', range(1, 1 + len(video_dataframe)))

- Calculating the time within the video by dividing the frame by the fps

In [ ]:
video_dataframe["video_time"] = video_dataframe["frame_number"] / fps

In [ ]:
video_dataframe["video_seconds"] = video_dataframe["video_time"].astype(int)

In [ ]:
video_dataframe

## Combining the ephys and video dataframe into one

In [ ]:
ephy_and_video_dataframe = pd.merge(ephys_dataframe, video_dataframe, left_on='voltage_time_stamp', right_on='PosTimestamp', how="left")

In [ ]:
ephy_and_video_dataframe

In [ ]:
ephy_and_video_dataframe.columns

- There are only a small number of rows that have information for the video, because the sampling rate is much smaller. 

In [ ]:
ephy_and_video_dataframe.dropna(subset=["PosTimestamp"])

- Filling in all the blank cells with the previous rows for the video related columns into new columns. This can be used to select for all rows that correspond to something happening within the video

In [ ]:
ephy_and_video_dataframe.columns

In [ ]:
for col in ephy_and_video_dataframe.columns:
    if "filled" not in col:
        ephy_and_video_dataframe['filled_{}'.format(col)] = ephy_and_video_dataframe[col].fillna(method='ffill')


In [ ]:
ephy_and_video_dataframe['filled_PosTimestamp'] = ephy_and_video_dataframe['PosTimestamp'].fillna(method='ffill')
ephy_and_video_dataframe['filled_frame_number'] = ephy_and_video_dataframe['frame_number'].fillna(method='ffill')
ephy_and_video_dataframe['filled_video_time'] = ephy_and_video_dataframe['video_time'].fillna(method='ffill')
ephy_and_video_dataframe['filled_video_seconds'] = ephy_and_video_dataframe['video_seconds'].fillna(method='ffill')

In [ ]:
ephy_and_video_dataframe.tail()

## Adding the DIN info

In [ ]:
DIN_dataframe = pd.DataFrame(tone_onset_DIN_state_array)

- Dropping the first two rows because that is when things start

In [ ]:
DIN_dataframe = DIN_dataframe.drop([0,1]).reset_index(drop=True)

In [ ]:
DIN_dataframe["trial_number"] = DIN_dataframe["state"].cumsum()

In [ ]:
DIN_dataframe

In [ ]:
ephy_and_video_dataframe = pd.merge(ephy_and_video_dataframe, DIN_dataframe, left_on='voltage_time_stamp', right_on='time', how="left")


In [ ]:
ephy_and_video_dataframe.head()

In [ ]:
tone_info_dataframe = ephy_and_video_dataframe.dropna(subset=["time"]).reset_index(drop=True)


In [ ]:
tone_info_dataframe["voltage_index"] = tone_info_dataframe["voltage_time_stamp"] - voltage_timestamp_array[0]

In [ ]:
tone_info_dataframe = tone_info_dataframe.dropna(axis="columns")
# To remove last tone light
tone_info_dataframe = tone_info_dataframe.iloc[0:-1]

In [ ]:
tone_info_dataframe

In [ ]:
results_dict = {1390826: 'rewarded',
 2990825: 'rewarded',
 4790823: 'rewarded',
 6390821: 'omission',
 7890820: 'rewarded',
 9890818: 'rewarded',
 11790816: 'rewarded',
 13590815: 'rewarded',
 15190821: 'omission',
 16990812: 'rewarded',
 18990809: 'rewarded',
 20790805: 'omission',
 23190805: 'rewarded',
 24990804: 'rewarded',
 30949197: 'win',
 32549196: 'win',
 34349195: 'win',
 35949193: 'win',
 37449192: 'win',
 39449187: 'win',
 41349188: 'win',
 43149186: 'win',
 44749185: 'win',
 46549183: 'win',
 48549181: 'win',
 50349180: 'win',
 52749175: 'win',
 54549173: 'win',
 56249171: 'win',
 58049170: 'win',
 59949171: 'win',
 62349168: 'win',
 63949167: 'win'}

In [ ]:
tone_info_dataframe["trial_type"] = tone_info_dataframe["voltage_index"].map(results_dict)

In [ ]:
tone_info_dataframe

In [ ]:
tone_info_dataframe[tone_info_dataframe["trial_type"] == "rewarded"]

In [ ]:
tone_info_dataframe[tone_info_dataframe["trial_type"] == "win"]

In [ ]:
recording_file_name = file_to_data["raw"]["raw_group0.dat"]["original_file"]

In [ ]:
recording_base_name = os.path.splitext(recording_file_name)[0]

In [ ]:
recording_base_name

In [ ]:
tone_info_dataframe.to_csv("./proc/{}.tone_timestamps.csv".format(recording_base_name))

In [ ]:
tone_info_dataframe

In [ ]:
raise ValueError()

In [ ]:
ephy_and_video_dataframe["filled_state"] = ephy_and_video_dataframe["state"].ffill()
ephy_and_video_dataframe["filled_trial_number"] = ephy_and_video_dataframe["trial_number"].ffill()

In [ ]:
ephy_and_video_dataframe.head()

In [ ]:
ephy_and_video_dataframe.tail()

In [ ]:
trial_1_df = ephy_and_video_dataframe[ephy_and_video_dataframe["filled_trial_number"] == 2]

In [ ]:
trial_1_df.head()

In [ ]:
file_to_data["raw"]

In [ ]:
ephy_and_video_dataframe.to_csv("./proc/{}.timestamps.csv".format(recording_base_name))

In [ ]:
raise ValueError()

- Original frame number(before light turns on)

In [ ]:
current_video_frame = trial_1_df["filled_frame_number"].min()

- Corrected frame number(that has the light on)

In [ ]:
corrected_video_frame = current_video_frame + 2

In [ ]:
cap.set(cv2.CAP_PROP_POS_FRAMES, corrected_video_frame)


In [ ]:
_, frame = cap.read()

In [ ]:
video_file_path

In [ ]:
output_directory = "./proc"

In [ ]:
output_directory

In [ ]:
os.makedirs(output_directory, exist_ok=True)

In [ ]:
video_file_basename = os.path.basename(video_file_path)

In [ ]:
video_file_root = os.path.splitext(video_file_basename)[0]

In [ ]:
video_file_root

In [ ]:
cv2.imwrite(os.path.join(output_directory, '{}.frame_{}.png'.format(video_file_root, corrected_video_frame)), frame)

# OTHER STUFF

## Getting the specific frame

In [ ]:
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print('Frame count:', frame_count)

- Original frame number(before light turns on)

In [ ]:
corrected_video_frame = current_video_frame

- Corrected frame number(that has the light on)

In [ ]:
corrected_video_frame = current_video_frame + 2

In [ ]:
cap.set(cv2.CAP_PROP_POS_FRAMES, corrected_video_frame)


In [ ]:
_, frame = cap.read()

In [ ]:
video_file_path

In [ ]:
output_directory = "./proc"

In [ ]:
output_directory

In [ ]:
os.makedirs(output_directory, exist_ok=True)

In [ ]:
video_file_basename = os.path.basename(video_file_path)

In [ ]:
video_file_root = os.path.splitext(video_file_basename)[0]

In [ ]:
video_file_root

In [ ]:
cv2.imwrite(os.path.join(output_directory, '{}.frame_{}.png'.format(video_file_root, corrected_video_frame)), frame)

- Where this video time stamp is(within the list of video time stamps), would be the frame number that corresponds to the ephys recording instance

In [ ]:
current_video_seconds = corrected_video_frame / fps

In [ ]:
current_video_seconds

In [ ]:
print("MED-PC signal is at {}:{}".format(int(current_video_seconds // 60), int(current_video_seconds % 60)))

- Array of Voltages for each channel

In [27]:
voltage_value_array = raw_recording_fields_text["data"]

In [28]:
voltage_value_array.shape

(67379591,)

In [29]:
voltage_value_array[0]

([-624, -357, -673, -728, -897, -807, -557, -588, -436, -571, -553, -677, -656,  -87, -342, -151,  262, -396, -661, -725, -281, -326, -493, -283, -512, -592, -580, -461, -432, -562, -442, -528],)

In [30]:
len(voltage_value_array[0][0])

32

In [31]:
voltage_value_array[:5]

array([([-624, -357, -673, -728, -897, -807, -557, -588, -436, -571, -553, -677, -656,  -87, -342, -151,  262, -396, -661, -725, -281, -326, -493, -283, -512, -592, -580, -461, -432, -562, -442, -528],),
       ([-475, -152, -545, -497, -736, -381, -327, -229, -217, -419, -282, -618, -350,  107, -150,   10,  509, -146, -459, -569,  -17,   34, -122,  -79, -196, -346, -297, -263, -186, -300, -160, -293],),
       ([-293,   38, -396, -300, -445, -162, -130,  -20,  -20, -186, -134, -511, -390,  326,   33,  223,  732,   33, -298, -362,   58,   72,   -1,   66,  -55,  -90,  -15,  -54,  -66, -142,  -99, -119],),
       ([-127,  283, -171, -185, -249,   42,  -98,   96,  119,  -85,  -36, -306, -202,  438,  197,  378,  871,  194,  -63, -260,  275,  201,  171,  190,   64,   15,    3,   47,   10,   31,   54,  -80],),
       ([  -1,  304, -166,  -61,  -63,  129,   34,  234,  135,   73,   50, -211, -202,  511,  275,  465,  929,  255,  165,  -52,  276,  332,  182,  347,   43,   50,  243,  131,   69,  